In [1]:
import pandas as pd
import matplotlib as plt


csv_file = "Resources/parking-citations.csv"
citations_df = pd.read_csv(csv_file)

C:\Users\brandon\Anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (0,7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
vd_list = citations_df["Violation Description"].dropna().unique().tolist()
descript = []

for _strng in vd_list:
    _string = str(_strng)
    if(_string[0].isdigit()):
        if(' ' in _string):
            descript.append(_string)
    else :
        descript.append(_strng)
        

In [3]:
descript.sort()

In [4]:
trimmed_df = citations_df[citations_df["Violation Description"].isin(descript)]

In [5]:
fines = trimmed_df['Fine amount'].dropna().unique().tolist()

In [6]:
trimmed_df = trimmed_df[trimmed_df['Fine amount'].isin(fines)]

In [8]:
trimmed_df['Fine amount'] = trimmed_df['Fine amount'].astype('int64')

In [11]:
trimmed_df['Violation Description'] = trimmed_df['Violation Description'].replace(
{  
  'ABAND VEH/HIGHWAY': 'ABANDONED VEHICLE',
  'ABAND VEH/PUB/PRIV': 'ABANDONED VEHICLE',
  'BLK BIKE PATH OR LAN': 'BLK BIKE PATH OR LANE',
  'BLK INT PROHIB': 'BLOCKING INTERSECTION',  
  'BLOCKING INTERSECTIO': 'BLOCKING INTERSECTION',
  'WITHIN INTERSECTION': 'BLOCKING INTERSECTION',
  'COMM VEH OVER TIME L':'COMM VEH OVER TIME LIMIT',
  'COMVEH RES/OV TM B-2':'COMM VEH OVER TIME LIMIT',
  'COMVEH RES/OV TM C-3':'COMM VEH OVER TIME LIMIT',
  'COMVEH RES/OV TM LMT':'COMM VEH OVER TIME LIMIT',
  'DEF WINDSHIELD': 'DEFECTIVE WIPERS',
  'DEFECT WIPERS' : 'DEFECTIVE WIPERS',
  'DISABLED PARKING/BOU': 'DISABLED PARKING/BOUNDARIES',
  'DISABLED PARKING/CRO': 'DISABLED PARKING/CROSS HATCH',
  'DISABLED PARKING/NO': 'DISABLED PARKING/NO DP ID',
  'DISABLED PARKING/OBS': 'DISABLED PARKING/OBSTRUCT ACCESS',
  'HANDICAP/CROSS HATCH':'DISABLED PARKING/CROSS HATCH',
  'HANDICAP/NO DP ID':'DISABLED PARKING/NO DP ID',
  'HANDICAP/ON LINE' :'DISABLED PARKING/BOUNDARIES',
  'HANDICPD/BLOCKING':'DISABLED PARKING/OBSTRUCT ACCESS',
  'BLOCK HANDICAP ACCES':'DISABLED PARKING/OBSTRUCT ACCESS',
  'DOUBLE PARK': 'DOUBLE PARKING',
  'EXCEED 72 HOURS': 'EXCEEDED TIME LIMIT',
  'EXCEED 72HRS-ST' : 'EXCEEDED TIME LIMIT',
  'EXCEED TIME LMT': 'EXCEEDED TIME LIMIT',
  'NO STOP/STAND AM': 'NO STOPPING/STANDING', 
  'NO STOP/STAND PM': 'NO STOPPING/STANDING',
  'STOP/STAND PROHB': 'NO STOPPING/STANDING',
  'STOP/STAND PROHIBIT': 'NO STOPPING/STANDING',
  'NO STOP/STAND': 'NO STOPPING/STANDING',
  'NO STOP/STANDING': 'NO STOPPING/STANDING',
  'TM LMT EXCEEDED': 'EXCEEDED TIME LIMIT',
  'TIME LIMIT/CITY LOT': 'EXCEEDED TIME LIMIT',
  'PARKED OVER TIME LIM': 'EXCEEDED TIME LIMIT',
  'PARKED OVER TIME LIMIT': 'EXCEEDED TIME LIMIT',
  'PK BYND TM LMTS': 'EXCEEDED TIME LIMIT',
  'FAIL DEP COIN': 'NO COIN DEP IN METER',
  'PREF PARKING': 'PREFERENTIAL PARKING',
  'NO PARK STRT CLEAN': 'NO PARK/STREET CLEAN',
  'CLEANING VEH/STREET': 'NO PARK/STREET CLEAN',
  'METER EXP.':  'METER EXPIRED',
  'NO PARKING BETWEEN P': 'NO PARKING BETWEEN POSTED HOURS',
  'NO PK BET 1-3AM' : 'NO PARKING BETWEEEN POSTED HOURS',
  'NO STOPPING/ANTI-GRI':'NO STOPPING/ANTI-GRIDLOCK ZONE',
  'PARK IN GRID LOCK ZN' : 'NO STOPPING/ANTI-GRIDLOCK ZONE',
  'LOAD/UNLOAD ONLY' : 'LOADING ZONES',
  'OFF STR/OVERTIME/MTR' : 'OFF STREET/OVERTIME',
  'OVERSIZED VEHICLE PA' : 'OVERSIZED VEHICLE PARKING',
  'OVERSIZED VEHICLE PARKING TOPHAM ST' : 'OVERSIZED VEHICLE PARKING',
  'OVNIGHT PRK W/OUT PE' : 'OVERNIGHT PARKING',
  'OVNIGHT PRK W/OUT PE' : 'RED FLAG DAY',
  'PK NR FIRE HYDR' : 'FIRE HYDRANT',
  'PARK RESTRCD': 'PK IN PROH AREA',
  'PARK-PSTD AREAS': 'PK IN PROH AREA',
  'PARKED IN BUS ZONE': 'PK IN PROH AREA',
  'PARKED IN CROSSWALK':'PK IN PROH AREA',
  'PARKED IN ELEC VEHICLE SPACE' : 'PK IN PROH AREA',
  'PARKED IN FIRE LANE': 'PK IN PROH AREA',
  'PARKED IN PARKWAY': 'PK IN PROH AREA',
  'PARKED ON BRIDGE': 'PK IN PROH AREA',
  'PARKED ON SIDEWALK': 'PK IN PROH AREA',
  'PARKING-SNOW AREA': 'PK IN PROH AREA',
  'PARKING/FRONT YARD': 'PK IN PROH AREA',
  'PARKING/FRONT YARD 1' : 'PK IN PROH AREA',
  'PARKING/FRONT YARD 2': 'PK IN PROH AREA',
  'PARKING/FRONT YARD 3' : 'PK IN PROH AREA',
  'PARKNG IN TUNNEL' : 'PK IN PROH AREA',
  'PK ON PRIV ST' : 'PK IN PROH AREA',
  'PK OTSD PSTD AR' : 'PK IN PROH AREA',
  'PK OUTSD PK STL' : 'PK IN PROH AREA',
  'PK OUTSD SPACE' : 'PK IN PROH AREA',
  'PRKG PUBL GRNDS' : 'PK IN PROH AREA',
  'PK OVR 2 SPACES' : 'PK OVR PNTD LNS',
  'SIGN POSTED - NO PAR' : 'NO PARKING',
  'SIGN PSTD-NO PK' : 'NO PARKING',
  'SIGN POSTED - NO PARKING' : 'NO PARKING',
  'YELLOW CURB' : 'YELLOW ZONE',
  'WHITE CURB' : 'WHITE ZONE',
  'RED CURB' : 'RED ZONE',
  'PKD IN/ON PARKWAY' : 'PK IN PROH AREA',
  'RESTRCTD ZONES': 'RESTRICTED ZONE',
  'RESTRICTED TAXI ZONE': 'RESTRICTED ZONE',
})






In [12]:
trimmed_df['Violation Description'].value_counts()

NO PARK/STREET CLEAN              2731040
METER EXPIRED                     1643530
RED ZONE                           713420
PREFERENTIAL PARKING               663702
DISPLAY OF TABS                    577836
NO PARKING                         420192
EXCEEDED TIME LIMIT                397966
NO STOPPING/STANDING               394810
DISPLAY OF PLATES                  261671
WHITE ZONE                         227859
PK IN PROH AREA                    156406
STANDNG IN ALLEY                   132570
BLOCKING DRIVEWAY                  118282
NO STOPPING/ANTI-GRIDLOCK ZONE     105161
YELLOW ZONE                         98176
NO EVIDENCE OF REG                  97200
18 IN. CURB/2 WAY                   83027
FIRE HYDRANT                        74596
DOUBLE PARKING                      63258
OUTSIDE LINES/METER                 56488
OFF STREET/OVERTIME                 49990
DISABLED PARKING/NO DP ID           41739
COMM VEH OVER TIME LIMIT            41707
EXPIRED TAGS                      

In [14]:
trimmed_df = trimmed_df.rename(columns={'Fine amount': 'Fine Amount', 'Violation code': 'Violation Code'})
trimmed_df.head(15)

,Ticket number,Issue Date,Issue time,Meter Id,Marked Time,RP State Plate,Plate Expiry Date,VIN,Make,Body Style,Color,Location,Route,Agency,Violation Code,Violation Description,Fine Amount,Latitude,Longitude
0,1103341116,2015-12-21T00:00:00,1251.0,NaN,NaN,CA,200304.0,NaN,HOND,PA,GY,13147 WELBY WAY,01521,1.0,4000A1,NO EVIDENCE OF REG,50,99999.0,99999.0
1,1103700150,2015-12-21T00:00:00,1435.0,NaN,NaN,CA,201512.0,NaN,GMC,VN,WH,525 S MAIN ST,1C51,1.0,4000A1,NO EVIDENCE OF REG,50,99999.0,99999.0
2,1104803000,2015-12-21T00:00:00,2055.0,NaN,NaN,CA,201503.0,NaN,NISS,PA,BK,200 WORLD WAY,2R2,2.0,8939,WHITE ZONE,58,6439997.9,1802686.4
4,1105461453,2015-09-15T00:00:00,115.0,NaN,NaN,CA,200316.0,NaN,CHEV,PA,BK,GEORGIA ST/OLYMPIC,1FB70,1.0,8069A,NO STOPPING/STANDING,93,99999.0,99999.0
5,1106226590,2015-09-15T00:00:00,19.0,NaN,NaN,CA,201507.0,NaN,CHEV,VN,GY,SAN PEDRO S/O BOYD,1A35W,1.0,4000A1,NO EVIDENCE OF REG,50,99999.0,99999.0
6,1106500452,2015-12-17T00:00:00,1710.0,NaN,NaN,CA,201605.0,NaN,MAZD,PA,BL,SUNSET/ALVARADO,00217,1.0,8070,NO STOPPING/ANTI-GRIDLOCK ZONE,163,99999.0,99999.0
7,1106500463,2015-12-17T00:00:00,1710.0,NaN,NaN,CA,201602.0,NaN,TOYO,PA,BK,SUNSET/ALVARADO,00217,1.0,8070,NO STOPPING/ANTI-GRIDLOCK ZONE,163,99999.0,99999.0
8,1106506402,2015-12-22T00:00:00,945.0,NaN,NaN,CA,201605.0,NaN,CHEV,PA,BR,721 S WESTLAKE,2A75,1.0,8069AA,NO STOPPING/STANDING,93,99999.0,99999.0
9,1106506413,2015-12-22T00:00:00,1100.0,NaN,NaN,CA,201701.0,NaN,NISS,PA,SI,1159 HUNTLEY DR,2A75,1.0,8069AA,NO STOPPING/STANDING,93,99999.0,99999.0
10,1106506424,2015-12-22T00:00:00,1100.0,NaN,NaN,CA,201511.0,NaN,FORD,TR,WH,1159 HUNTLEY DR,2A75,1.0,8069AA,NO STOPPING/STANDING,93,99999.0,99999.0
